In [91]:
import re
from typing import Optional
import pandas as pd
import asyncio
from playwright.async_api import async_playwright
import asyncio
from typing import List, Tuple
from playwright.async_api import async_playwright

In [ ]:
results_actual = []

In [168]:
async def get_section_links(url: str, section_number: str) -> List[Tuple[str,str]]:
    """
    Given a delcode URL (e.g. “…/#6091”) and its numeric id (e.g. "6091"),
    return all (href, text) for <a> inside that Section’s div.
    """
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page    = await browser.new_page()
        await page.goto(url, wait_until="domcontentloaded")

        links = await page.evaluate(f"""
        () => {{
          // 1) find the <div class="SectionHead" id="{section_number}">
          const head = document.querySelector("div.SectionHead[id='{section_number}']");
          if (!head) return [];
          // 2) find its parent <div class="Section">
          const sectionDiv = head.closest("div.Section");
          if (!sectionDiv) return [];
          // 3) pull every <a> inside that Section
          return Array.from(sectionDiv.querySelectorAll("a"))
                      .map(a => [a.href, a.textContent.trim()]);
        }}
        """)

        await browser.close()
        return links

async def fetch_chapter_body_text(url: str) -> str:
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page    = await browser.new_page()
        await page.goto(url, wait_until="domcontentloaded")
        # wait for the main content container
        await page.wait_for_selector("div#chapterBody")
        # grab its entire inner text
        text = await page.inner_text("div#chapterBody")
        await browser.close()
        return text
        
def extract_last_approval_year(document_text: str) -> Optional[int]:
    """
    Finds all occurrences of lines like:
      • “Approved June 30, 2021”
      • “Approved April 5, A. D. 1909.”
      • “Passed at Dover, February 18, 1889.”
      • “Approved June 30. 1988.”
    and returns the YEAR from the last one found. Returns None if no match.
    """
    pattern = r"""
      (?:Approved|Passed(?:\s+at\s+[^,]+)?)   # “Approved” or “Passed at LOCATION”
      ,?\s*                                   # optional comma + whitespace
      [A-Za-z]+                               # Month name
      \s+\d{1,2}                              # Day
      (?:[.,]\s*)                             # comma or period after the day
      (?:A\.\s*D\.\s*)?                       # optional “A. D.” prefix
      (\d{4})                                 # capture the 4-digit year
    """
    matches = list(re.finditer(pattern, document_text, flags=re.IGNORECASE | re.VERBOSE))
    if not matches:
        return None
    return int(matches[-1].group(1))

for d in docs:
    print(d, "→", extract_last_approval_year(d))
# Outputs:
# … Approved June 30. 1988. → 1988
# … Passed at Dover, February 18, 1889. → 1889
# … Approved April 5, A. D. 1909. → 1909
# … Approved June 30, 2021. … Approved July 1, 2022. → 2022



async def scrape_energy_sections() -> list[tuple[str, str]]:
    results: list[tuple[str,str]] = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page(viewport={"width":1280,"height":800})

        # 1) Go & search “energy”
        await page.goto("https://delcode.delaware.gov/", wait_until="networkidle")
        await page.fill("input#srch-term", "energy")
        await asyncio.gather(
            page.click("button[onclick='submitSearch()']"),
            page.wait_for_load_state("networkidle")
        )

        page_num = 1
        while True:
            print(f"[+] Page {page_num} results…")
            await page.wait_for_selector("#SearchResultsGrid tbody tr.k-master-row", timeout=15000)
            rows = page.locator("#SearchResultsGrid tbody tr.k-master-row")
            count = await rows.count()
            print(f"    → {count} statutes on this page")

            for i in range(count):
                row = rows.nth(i)
                # grab all gridcells
                cells = row.locator("td[role='gridcell']")
                n_cells = await cells.count()

                if n_cells >= 4:
                    # the 4th gridcell always holds the Section link
                    cell4 = cells.nth(3)
                    link  = cell4.locator("a").first

                    url   = await link.get_attribute("href", timeout=5000)
                    text  = (await link.inner_text()).strip()
                    entry = (url, text)

                    print(f"    → Saved: {entry}")
                    results.append(entry)
                    results_actual.append(entry)
                else:
                    print(f"    ! row {i} only has {n_cells} gridcells; skipping")

            # check if “Next” is disabled
            disabled = await page.locator("li.k-pager-nav.k-state-disabled").count() > 0
            if disabled:
                print("[+] Next is disabled. Reached last page.")
                break

            # otherwise click Next
            next_btn = page.locator("a.k-link.k-pager-nav[title='Go to the next page']")
            print(f"    → Clicking Next → (page {page_num+1})")
            await asyncio.gather(
                next_btn.first.click(),
                page.wait_for_selector("#SearchResultsGrid tbody tr.k-master-row", timeout=15000)
            )
            page_num += 1

        await browser.close()

    return results

In [ ]:
# --- Run in Jupyter ---
sections = await scrape_energy_sections()
df_sections = pd.DataFrame(sections, columns=["url", "section_text"])

In [5]:
results = list(set(results_actual))
# turn into a DataFrame
df = pd.DataFrame(results, columns=['url', 'section_text'])
df['section_number'] = df['section_text'].str.replace(r'(?i)section\s*', '', regex=True)

In [8]:
links = []
# we iterate over (url, section_number) pairs directly, with a progress bar
for url, sec_num in tqdm(zip(df['url'], df['section_number']),
                         total=len(df),
                         desc="Fetching section links"):
    try:
        resultings = await get_section_links(url, sec_num)
        #print(resultings)
        links.append(resultings)
    except Exception as e:
        # you can log e or just skip
        print(f"⚠️ Error at {sec_num}: {e}")
        links.append([])  # keep indices aligned

# and finally attach to your DataFrame
df['section_links'] = links
urls = [url for sub in links for url, _ in sub]
urls = list(set(urls))

Fetching section links:   0%|          | 0/106 [00:00<?, ?it/s]

[['https://legis.delaware.gov/SessionLaws?volume=84&chapter=141', '84 Del. Laws, c. 141,\n                §\n              1']]
[['https://legis.delaware.gov/SessionLaws?volume=77&chapter=144', '77 Del. Laws, c. 144,\n                §\n              \u20021'], ['https://legis.delaware.gov/SessionLaws?volume=70&chapter=186', '70 Del. Laws, c. 186,\n                §\n              \u20021'], ['https://legis.delaware.gov/SessionLaws?volume=77&chapter=454', '77 Del. Laws, c. 454,\n                §§\n              \u20021, 3'], ['https://legis.delaware.gov/SessionLaws?volume=82&chapter=208', '82 Del. Laws, c. 208,\n                §\n              1']]
[['https://legis.delaware.gov/SessionLaws?volume=81&chapter=402', '81 Del. Laws, c. 402,\n                §\n              1'], ['https://legis.delaware.gov/SessionLaws?volume=83&chapter=91', '83 Del. Laws, c. 91,\n                §\n              1'], ['https://legis.delaware.gov/SessionLaws?volume=83&chapter=283', '83 Del. Laws, c. 283,\

In [174]:
# Example usage:
docs = []
for i in range(len(urls)):
    print(i)
    text = await fetch_chapter_body_text(urls[i])
    docs.append(text)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [180]:
years = []
for i in range(len(docs)):
    year = extract_approval_year(docs[i])
    years.append(year)

In [182]:
years

[2021,
 1971,
 2009,
 2023,
 1999,
 1984,
 2010,
 1997,
 1994,
 1982,
 1885,
 2002,
 None,
 2020,
 1983,
 1909,
 2015,
 2007,
 1997,
 1980,
 1990,
 1949,
 2005,
 2001,
 1943,
 2017,
 1992,
 1949,
 1990,
 2022,
 1955,
 2001,
 1990,
 2013,
 1905,
 None,
 None,
 2018,
 1951,
 1885,
 1903,
 1987,
 1901,
 1984,
 1897,
 2022,
 1999,
 2000,
 2003,
 1939,
 2024,
 1951,
 1984,
 2019,
 2024,
 2023,
 2021,
 2011,
 2014,
 2002,
 2021,
 2023,
 2008,
 2019,
 1893,
 1951,
 1915,
 2016,
 1941,
 1913,
 1937,
 2018,
 1980,
 2010,
 1909,
 1998,
 2009,
 2010,
 1977,
 2015,
 2002,
 2024,
 2024,
 1982,
 2009,
 2017,
 2022,
 1984,
 1909,
 2004,
 1991,
 2019,
 2002,
 1999,
 2023,
 1885,
 1980,
 1981,
 None,
 1991,
 2010,
 1999,
 2014,
 1945,
 2010,
 2021,
 2024,
 2000,
 1997,
 1997,
 2017,
 1973,
 1871,
 1980,
 2010,
 1911,
 1909,
 1963,
 2004,
 2001,
 1981,
 1883,
 1935,
 1871,
 2002,
 1979,
 1994,
 1974,
 2011,
 1873,
 1988,
 1947,
 1986,
 1996,
 2024,
 None,
 1990,
 2022,
 2007,
 1968,
 2000,
 2008,
 1883,

In [188]:
df_final = pd.DataFrame({'text': docs, 'year':years})

In [192]:
df_final['location'] = 'Delaware'
df_final.to_csv("delaware_energy_data.csv", index=False)